# Extract Template ROI
This notebook extracts the ROI of the template using template matching and the templates provided under the `templates` folder. The output is a CSV file with each image path, the corresponding template path, and the ROI of the template within the image.

In [ ]:
import sys
import math
from pathlib import Path
from typing import Dict, List

import cv2
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from bananas.utils import images

# Root path of project relative to this notebook
ROOT = Path('..')

sys.path.insert(1, str(ROOT / 'scripts'))
from datamodels import *
from utils import *

### Load all templates from the templates folder

In [ ]:
templates = tpl_load_all(ROOT / 'templates')
assert len(templates) > 0, 'Templates folder is empty, please run module #0'

### Use template matching to find the template in each of the source images

In [ ]:
def find_tpl(img: np.ndarray, tpl: Template):
    # Find template within the image
    res = cv2.matchTemplate(img, tpl.image, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    # Find the bounding box of template in image
    left, top = max_loc
    right, bottom = left + tpl.width, top + tpl.height
    
    return Box(left = left, top = top, right = right, bottom = bottom)

In [ ]:
# Iterate over all image drawings
tpl_roi_records = []
for tpl in tqdm(templates, leave=False):
    imdir = ROOT / 'drawings' / tpl.name.upper()
    impaths = [impath for impath in imdir.iterdir() if impath.suffix in IMG_EXTENSIONS]
    
    # Extract template ROIs from images
    for impath in impaths:
        relpath = str(impath.relative_to(ROOT))
        img = images.open_image(str(impath), convert='L')
        try:
            tpl_box = find_tpl(img, tpl)
            tpl_roi_records.append({
                'image_path': str(relpath),
                'template_name': tpl.name,
                'template_path': str(tpl.path.relative_to(ROOT)),
                'template_box': str(tpl_box)})
        except:
            tqdm.write('Template not found for %s' % impath)

print('ROIs processed: %d' % len(tpl_roi_records))

### Write results in a CSV file

In [ ]:
pd.DataFrame.from_records(tpl_roi_records).set_index('image_path').to_csv(ROOT / 'datasets' / 'template_roi.csv')